<div style="float: left; width: 90%; height: auto; line-height: 200px; vertical-align: middle; background: #000099; padding: 0; margin: 0; border-top-left-radius: 10px; border-bottom-left-radius: 10px; border-top-right-radius: 10px; border-bottom-right-radius: 10px; font-family: Calibri; font-size: 50px; overflow: hidden; text-align: center;">
        <div style="text-align: center; width: 80%; margin: 0 auto; overflow: hidden; line-height: 80px; background-color:#000099">
                  <span id="dept_name" class="pretty_span">
                          <font color="white">**Parameter Grid**</font>
                  </span>
        </div>
</div>

<div style="float: left; width: 90%; height: auto; line-height: 200px; vertical-align: middle; background: #000000; padding: 0; margin: 0; border-top-left-radius: 10px; border-bottom-left-radius: 10px; border-top-right-radius: 10px; border-bottom-right-radius: 10px; font-family: Calibri; font-size: 30px; overflow: hidden; text-align: center;">
        <div style="text-align: center; width: 80%; margin: 0 auto; overflow: hidden; line-height: 50px; background-color:#000000">
                  <span id="dept_name" class="pretty_span">
                          <font color="white">**Table of contents**</font>
                  </span>
        </div>
</div>

## 1. [Import Libraries](#Import Libraries)
## 2. [Load the Dataset](#Load the Dataset)
## 3. [Create the Train and Test Datasets](#Create the Train and Test Datasets)
## 4. [Define Tokenizer with Vocab Size](#Define Tokenizer with Vocab Size)
## 5. [Create the Model](#Create the Model)
## 6. [Reproductibility and Model Definition](#Reproductibility and Model Definition)
## 7. [Create and Tune the Parameter Grip](#Create and Tune the Parameter Grip)
## 8. [Get the results of all Models and Find the Best One](#Get the results of all Models and Find the Best One)

# <a id='Import Libraries'></a> <u><font color='orange'>Import Libraries</font></u>

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import metrics
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.layers import Flatten

from nltk import word_tokenize, pos_tag, chunk
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from pprint import pprint
import pandas as pd
import numpy as np

from keras import optimizers
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.constraints import maxnorm
from keras.layers import Dropout
import os

# <a id='Load the Dataset'></a> <u><font color='orange'>Load the Dataset</font></u>

In [ ]:
path = '/Users/leonidas/Documents/GitHub/Deep-Learning-Project/Results/'
filename = '25_cleaned_job_descriptions.csv'
data = pd.read_csv(os.path.join(path, filename), header = 0, names = ['Query', 'Description'])

# <a id='Create the Train and Test Datasets'></a> <u><font color='orange'>Create the Train and Test Datasets</font></u>

In [ ]:
#Split data to train and test (80 - 20)

train, test = train_test_split(data, test_size = 0.2)

train_descs = train['Description']
train_labels = train['Query']
#train_labels = train['Job Title']
 
test_descs = test['Description']
test_labels = test['Query']

# <a id='Define Tokenizer with Vocab Size'></a> <u><font color='orange'>Define Tokenizer with Vocab Size</font></u>

In [ ]:
num_labels = len(train_labels.unique().tolist())
vocab_size = 1000
 
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_descs)
 
x_train = tokenizer.texts_to_matrix(train_descs, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_descs, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

# <a id='Create the Model'></a> <u><font color='orange'>Create the Model</font></u>

In [ ]:
def create_model(optimizer='adam', init_mode='uniform', activation='relu', dropout_rate=0.0, weight_constraint=0, neurons=1):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(vocab_size,), activation = activation, kernel_initializer = init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(512, kernel_initializer = init_mode, activation= activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer, # or 'sgd'
              metrics = [metrics.categorical_accuracy, 'accuracy'])
    return model

# <a id='Reproductibility and Model Definition'></a> <u><font color='orange'>Reproductibility and Model Definition</font></u>

In [ ]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Define the model
model = KerasClassifier(build_fn=create_model, verbose=0)

# <a id='Create and Tune the Parameter Grip'></a> <u><font color='orange'>Create and Tune the Parameter Grip</font></u>

In [ ]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
epochs = [30, 50, 100]
batch_size = [16, 32, 64, 100, 200]
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
neurons = [512, 1024, 2048, 4096]
param_grid = dict(optimizer=optimizer, init_mode=init_mode, epochs=epochs, batch_size=batch_size, activation=activation, dropout_rate=dropout_rate, weight_constraint=weight_constraint, neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)

# <a id='Get the results of all Models and Find the Best One'></a> <u><font color='orange'>Get the results of all Models and Find the Best One</font></u>

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))